# Proyecto Integrador 1
---
Presentado por:
* Karla Orozco
* Jonathan zapata
* Juan Fernando Gallego

# Raw -> Trusted

### Imports and Instances

In [1]:
# Import common libraries:
import sys
# from awsglue.transforms import *
# from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
# from awsglue.context import GlueContext
# from awsglue.job import Job


In [3]:

## @params: [JOB_NAME]
# args = getResolvedOptions(sys.argv, ['JOB_NAME'])

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]") \
                    .appName('SparkByExamples.com') \
                    .getOrCreate()

# sc = SparkContext.getOrCreate()
# glueContext = GlueContext(sc)
# spark = glueContext.spark_session
# job = Job(glueContext)
# job.init(args['JOB_NAME'], args)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/04 20:11:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/04 20:11:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Read data from Glue Catalog

In [4]:
## @type: DataSource
## @args: [database = "pi1-kjj-trusted", table_name = "company1_csv", transformation_ctx = "DataSource0"]
## @return: company_dynamicframe
## @inputs: []
# company_dynamicframe = glueContext.create_dynamic_frame.from_catalog(
#        database = "pi1-kjj-trusted",
#        table_name = "company_csv",
#        transformation_ctx = "company_dynamicframe")
company_dynamicframe = spark.read.csv('raw/Company.csv', header='true', inferSchema='true')
company_dynamicframe.printSchema()

# company_tweet_dynamicframe = glueContext.create_dynamic_frame.from_catalog(
#        database = "pi1-kjj-trusted",
#        table_name = "company_tweet_csv")
company_tweet_dynamicframe = spark.read.csv('raw/Company_Tweet.csv', header='true', inferSchema='true')
company_tweet_dynamicframe.printSchema()

# tweet_dynamicframe = glueContext.create_dynamic_frame.from_catalog(
#        database = "pi1-kjj-trusted",
#        table_name = "tweet_csv")
tweet_dynamicframe = spark.read.csv('raw/Tweet.csv', header='true', inferSchema='true')
tweet_dynamicframe.printSchema()

# company_dataframe = company_dynamicframe.toDF()
# tweet_dataframe = tweet_dynamicframe.toDF()
# company_tweet_dataframe = company_tweet_dynamicframe.toDF()

company_dataframe = company_dynamicframe
tweet_dataframe = tweet_dynamicframe
company_tweet_dataframe = company_tweet_dynamicframe


root
 |-- ticker_symbol: string (nullable = true)
 |-- company_name: string (nullable = true)



root
 |-- tweet_id: long (nullable = true)
 |-- ticker_symbol: string (nullable = true)



root
 |-- tweet_id: long (nullable = true)
 |-- writer: string (nullable = true)
 |-- post_date: integer (nullable = true)
 |-- body: string (nullable = true)
 |-- comment_num: string (nullable = true)
 |-- retweet_num: string (nullable = true)
 |-- like_num: string (nullable = true)



### Summarize total engagement:

In [5]:

# Summarize total engagement:
# tweet_dataframe['total_engagement'] = tweet_dataframe['comment_num'] + tweet_dataframe['retweet_num'] + tweet_dataframe['like_num']
tweet_dataframe = tweet_dataframe.withColumn('total_engagement', sum(tweet_dataframe[col] for col in ['comment_num', 'retweet_num', 'like_num']))

# Get iso date (Not epoch)
tweet_dataframe = tweet_dataframe.withColumnRenamed('post_date', 'post_datetime_epoch')
# tweet_dataframe['post_datetime'] = pd.to_datetime(tweet_dataframe['post_datetime_epoch'],unit='s')
tweet_dataframe.head(2)

[Row(tweet_id=550441509175443456, writer='VisualStockRSRC', post_datetime_epoch=1420070457, body='lx21 made $10,008  on $AAPL -Check it out! http://profit.ly/1MnD8s?aff=202 Learn #howtotrade http://bit.ly/1c1NljX $EXE $WATT $IMRS $CACH $GMO', comment_num='0', retweet_num='0', like_num='1', total_engagement=1.0),
 Row(tweet_id=550441672312512512, writer='KeralaGuy77', post_datetime_epoch=1420070496, body='Insanity of today weirdo massive selling. $aapl bid up 45 cents after hours after non stop selling in trading hours', comment_num='0', retweet_num='0', like_num='0', total_engagement=0.0)]

### Get only date from datetime

In [6]:

from pyspark.sql import functions as f
# https://stackoverflow.com/a/54340652
tweet_dataframe = tweet_dataframe.withColumn('post_datetime', f.to_timestamp(tweet_dataframe.post_datetime_epoch))
# tweet_dataframe['post_date'] = tweet_dataframe['post_datetime'].dt.date
tweet_dataframe = tweet_dataframe.withColumn('post_date', f.to_date(tweet_dataframe.post_datetime))
#tweet_dataframe = tweet_dataframe.withColumn('post_date', f.to_date(f.col('post_datetime')))


### Get ticker symbol for each tweet

In [7]:
# Join company tweets with tweets dataframe:
# ticker_symbol_group = company_tweet_dataframe.groupBy('tweet_id')['ticker_symbol'].distinct()
# ticker_symbol_group.rename("ticker_symbol_group", inplace=True)
ticker_symbol_group = company_tweet_dataframe.groupBy('tweet_id').agg(
    f.collect_set(f.col('ticker_symbol')).alias('ticker_symbol_group')
)
# print(ticker_symbol_group.show(n=5))

tweet_dataframe = tweet_dataframe.join(ticker_symbol_group, tweet_dataframe['tweet_id'] == ticker_symbol_group['tweet_id'], 'left')
# print(tweet_dataframe.show(truncate=False, n=2))


### Tokenize & Lemmatize

In [8]:
from pyspark.ml.feature import NGram, Tokenizer, StopWordsRemover
from pyspark.ml import Pipeline

In [9]:
#tweet_head['body_tokenized'] = tweet_head.apply(lambda row: nltk.word_tokenize(str(row['body'])), axis=1)
#tweet_dataframe = tweet_dataframe.withColumn('body_tokenized', nltk.word_tokenize(str(col)) for col in ['body'])
# tweet_dataframe = tweet_dataframe.withColumn('body_tokenized', tweet_dataframe.select('body').rdd.map(lambda x: 
#     TweetTokenizer(str(x))).collect()
# )
# print(tweet_dataframe.show(n=5))

# stop_words_nltk = set(stopwords.words('english'))

# https://towardsdatascience.com/sentiment-analysis-with-pyspark-bc8e83f80c35

tokenizer = Tokenizer(inputCol="body", outputCol="body_tokenized")
remove_stop_words = StopWordsRemover(inputCol="body_tokenized", outputCol="no_stopwords")
unigrammer = NGram(n=1, inputCol="no_stopwords", outputCol="ngrams") 


data_prep_pipe = Pipeline(
    stages=[tokenizer, remove_stop_words, unigrammer]
)


transformed = data_prep_pipe.fit(tweet_dataframe).transform(tweet_dataframe)

21/12/04 20:12:08 WARN StopWordsRemover: Default locale set was [es_419]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


In [10]:
#transformed.select(['body', 'ngrams']).show()

transformed.printSchema()


root
 |-- tweet_id: long (nullable = true)
 |-- writer: string (nullable = true)
 |-- post_datetime_epoch: integer (nullable = true)
 |-- body: string (nullable = true)
 |-- comment_num: string (nullable = true)
 |-- retweet_num: string (nullable = true)
 |-- like_num: string (nullable = true)
 |-- total_engagement: double (nullable = true)
 |-- post_datetime: timestamp (nullable = true)
 |-- post_date: date (nullable = true)
 |-- tweet_id: long (nullable = true)
 |-- ticker_symbol_group: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- body_tokenized: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- no_stopwords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ngrams: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [11]:
from pyspark.sql.functions import concat_ws
transformed = transformed.withColumn('bow', concat_ws(' ', 'ngrams'))

transformed.select('bow').show()

+--------------------+
|                 bow|
+--------------------+
|lx21 made $10,008...|
|insanity today we...|
|s&p100 #stocks pe...|
|$gm $tsla: volksw...|
|swing trading: 8....|
|swing trading: 8....|
|swing trading: 8....|
|swing trading: 8....|
|swing trading: 8....|
|swing trading: 8....|
|swing trading: 8....|
|$unp $orcl $qcom ...|
|$aapl apple goes ...|
|“@wsj: apple sued...|
|apple filed stylu...|
|@cnbc 15 top #tra...|
|searched hundreds...|
|top 10 searched #...|
|2014 year review ...|
|give brain workou...|
+--------------------+
only showing top 20 rows



### Sentiment Analysis

In [16]:
from pyspark.sql.functions import udf
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/jonny/nltk_data...


In [26]:

@udf
def sentiment(x):
  import os
  os.environ['NLTK_DATA'] = '/tmp'
  import nltk
  nltk.data.path.append('/tmp')
  print('nltk.data.path')
  print(nltk.data.path)
  # nltk.downloader.download('vader_lexicon')
  nltk.download('vader_lexicon', download_dir = '/tmp') # For Sentyment Analysis
  return SentimentIntensityAnalyzer().polarity_scores(x)['compound']

# sentiment = udf(lambda x: SentimentIntensityAnalyzer().polarity_scores(x)['compound'])
# spark.udf.register('sentiment', sentiment)
transformed = transformed.withColumn('score_vader', sentiment('bow').cast('double'))


21/12/04 18:36:15 WARN SimpleFunctionRegistry: The function sentiment replaced a previously registered function.


In [ ]:
transformed.select('score_vader').show()

In [ ]:

# conditions = [
#     (tweet["score_vader"] >= .05),
#     (tweet["score_vader"] > -.05) & (tweet["score_vader"] < .05),
#     (tweet["score_vader"] <= -.05),
# ]

# choices = ['positive', 'neutral', 'negative']
# tweet['sentiment'] = np.select(conditions, choices)

@udf
def scorevader_classifier(s):
  if (s >= 0.05): return 'positive'
  elif s <= -0.05: return 'neutral'
  else: return 'negative'

transformed = transformed.withColumn('sentiment', scorevader_classifier('score_vader'))


### Split data by ticker symbol

In [13]:
# company_dataframe.show(n=6)

+-------------+------------+
|ticker_symbol|company_name|
+-------------+------------+
|         AAPL|       apple|
|         GOOG|  Google Inc|
|        GOOGL|  Google Inc|
|         AMZN|  Amazon.com|
|         TSLA|   Tesla Inc|
|         MSFT|   Microsoft|
+-------------+------------+



In [17]:
# def split_data(x, v_split):
#     #columna de listas de tickers
#     t=x['ticker_symbol_group']
#     #vector de boleanos
#     v_bool=t.apply(lambda tikers: np.isin(v_split,tikers))
#     #retornar datos filtrados
#     return x.loc[v_bool].copy()
from pyspark.sql.functions import array_contains, col

AAPL = transformed \
  .filter(array_contains(col('ticker_symbol_group'), 'AAPL'))

AAPL.show()

+------------------+---------------+-------------------+--------------------+-----------+-----------+--------+----------------+-------------------+----------+------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|          tweet_id|         writer|post_datetime_epoch|                body|comment_num|retweet_num|like_num|total_engagement|      post_datetime| post_date|          tweet_id|ticker_symbol_group|      body_tokenized|        no_stopwords|              ngrams|                 bow|
+------------------+---------------+-------------------+--------------------+-----------+-----------+--------+----------------+-------------------+----------+------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|550447574285418497|      btcgemini|         1420071903|We searched throu...|          0|          0|       0|             0.0|2014-12-31 19:25:03|2014-12

In [16]:
GOOG = transformed \
  .filter(array_contains(col('ticker_symbol_group'), 'GOOG'))

GOOG.show()


+------------------+---------------+-------------------+--------------------+-----------+-----------+--------+----------------+-------------------+----------+------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|          tweet_id|         writer|post_datetime_epoch|                body|comment_num|retweet_num|like_num|total_engagement|      post_datetime| post_date|          tweet_id|ticker_symbol_group|      body_tokenized|        no_stopwords|              ngrams|                 bow|
+------------------+---------------+-------------------+--------------------+-----------+-----------+--------+----------------+-------------------+----------+------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|550461555423584257|     t_nathan95|         1420075236|Prediction: $TWTR...|          0|          0|       1|             1.0|2014-12-31 20:20:36|2014-12

In [ ]:


GOOGL = transformed \
  .filter(array_contains(col('ticker_symbol_group'), 'GOOGL'))

GOOGL.show()



In [ ]:

AMZN = transformed \
  .filter(array_contains(col('ticker_symbol_group'), 'AMZN'))

AMZN.show()


In [ ]:


TSLA = transformed \
  .filter(array_contains(col('ticker_symbol_group'), 'TSLA'))

TSLA.show()


In [ ]:


MSFT = transformed \
  .filter(array_contains(col('ticker_symbol_group'), 'MSFT'))

MSFT.show()

### Store to S3

In [ ]:
# Export data to S3:

from awsglue.dynamicframe import DynamicFrame
AAPL_dyf = DynamicFrame.fromDF(AAPL, glueContext, 'AAPL')
GOOG_dyf = DynamicFrame.fromDF(GOOG, glueContext, 'GOOG')
GOOGL_dyf = DynamicFrame.fromDF(GOOGL, glueContext, 'GOOGL')
AMZN_dyf = DynamicFrame.fromDF(AMZN, glueContext, 'AMZN')
TSLA_dyf = DynamicFrame.fromDF(TSLA, glueContext, 'TSLA')
MSFT_dyf = DynamicFrame.fromDF(MSFT, glueContext, 'MSFT')

glueContext.write_dynamic_frame.from_options(
       frame = AAPL_dyf,
       connection_type = 's3',
       connection_options = {'path': 's3://pi1-kjj/trusted/AAPL'},
       format = 'parquet')
glueContext.write_dynamic_frame.from_options(
       frame = GOOG_dyf,
       connection_type = 's3',
       connection_options = {'path': 's3://pi1-kjj/trusted/GOOG'},
       format = 'parquet')
glueContext.write_dynamic_frame.from_options(
       frame = GOOGL_dyf,
       connection_type = 's3',
       connection_options = {'path': 's3://pi1-kjj/trusted/GOOGL'},
       format = 'parquet')
glueContext.write_dynamic_frame.from_options(
       frame = AMZN_dyf,
       connection_type = 's3',
       connection_options = {'path': 's3://pi1-kjj/trusted/AMZN'},
       format = 'parquet')
glueContext.write_dynamic_frame.from_options(
       frame = TSLA_dyf,
       connection_type = 's3',
       connection_options = {'path': 's3://pi1-kjj/trusted/TSLA'},
       format = 'parquet')
glueContext.write_dynamic_frame.from_options(
       frame = MSFT_dyf,
       connection_type = 's3',
       connection_options = {'path': 's3://pi1-kjj/trusted/MSFT'},
       format = 'parquet')

print('End storing to S3')
